In [ ]:
import zipfile

!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
zip_ref=zipfile.ZipFile("10_food_classes_10_percent.zip","r")
zip_ref.extractall()
zip_ref.close()

--2022-12-07 08:09:01--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.4.128, 74.125.24.128, 172.217.194.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.4.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip.1’

10_food_classes_10_ 100%[===================>] 160.74M  23.1MB/s    in 7.9s    

2022-12-07 08:09:09 (20.3 MB/s) - ‘10_food_classes_10_percent.zip.1’ saved [168546183/168546183]



In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir="10_food_classes_10_percent/train"
test_dir="10_food_classes_10_percent/test"

train_datagen=ImageDataGenerator(rescale=1/255,
                                 )
test_datagen=ImageDataGenerator(rescale=1/255,
                                 )

train_data= train_datagen.flow_from_directory(directory=train_dir,
                                              target_size=(300,300),
                                             )
test_data= test_datagen.flow_from_directory(directory=test_dir,
                                            target_size=(300,300),
                                           )

Found 750 images belonging to 10 classes.
Found 2500 images belonging to 10 classes.


In [ ]:
import datetime
def tensorboard_callback(dir_name, exp_name):
  dir_log=dir_name+"/"+exp_name+"/"+str(datetime.datetime.now())
  tensorboard_callback=tf.keras.callbacks.TensorBoard(log_dir=dir_log)
  print("saving TensorBoard files to",dir_log)
  return tensorboard_callback

In [ ]:
efficientnetb0="https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"
efficientnetb7="https://tfhub.dev/tensorflow/efficientnet/b7/feature-vector/1"
resnet="https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/5"

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

def model_creator(model_url, num_class=10):
  feature_extractor=hub.KerasLayer(model_url,
                                    trainable= False,
                                    )
  model=tf.keras.Sequential([ feature_extractor,
                              tf.keras.layers.Dense(num_class, activation="softmax"),
                            ])

  return model

In [ ]:
model_eb0=model_creator(efficientnetb0, 10)
model_eb0.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
            optimizer=tf.keras.optimizers.Adam(),
            metrics=["accuracy"],
            )
history_eb0=model_eb0. fit(train_data,
            epochs=10,
            batch_size=32,
            validation_data=(test_data),
            callbacks=[tensorboard_callback("tensorboard", "efficientnet_b0")]
            )

saving TensorBoard files to tensorboard/efficientnet_b0/2022-12-07 08:09:29.092323
Epoch 1/10
24/24 [==============================] - 29s 883ms/step - loss: 1.9500 - accuracy: 0.3880 - val_loss: 1.4211 - val_accuracy: 0.6896
Epoch 2/10
24/24 [==============================] - 19s 795ms/step - loss: 1.1843 - accuracy: 0.7480 - val_loss: 0.9565 - val_accuracy: 0.7980
Epoch 3/10
24/24 [==============================] - 19s 814ms/step - loss: 0.8449 - accuracy: 0.8253 - val_loss: 0.7586 - val_accuracy: 0.8344
Epoch 4/10
24/24 [==============================] - 18s 791ms/step - loss: 0.6750 - accuracy: 0.8667 - val_loss: 0.6533 - val_accuracy: 0.8508
Epoch 5/10
24/24 [==============================] - 18s 788ms/step - loss: 0.5713 - accuracy: 0.8867 - val_loss: 0.5895 - val_accuracy: 0.8564
Epoch 6/10
24/24 [==============================] - 18s 783ms/step - loss: 0.4988 - accuracy: 0.9080 - val_loss: 0.5446 - val_accuracy: 0.8596
Epoch 7/10
24/24 [==============================] - 18s 788

In [ ]:
model_eb7=model_creator(efficientnetb7, 10)
model_eb7.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
            optimizer=tf.keras.optimizers.Adam(),
            metrics=["accuracy"],
            )
history_eb7=model_eb7. fit(train_data,
            epochs=10,
            batch_size=32,
            validation_data=(test_data),
            callbacks=[tensorboard_callback("tensorboard", "efficientnet_b7")]
            )

saving TensorBoard files to tensorboard/efficientnet_b7/2022-12-07 08:13:41.338526
Epoch 1/10
24/24 [==============================] - 105s 3s/step - loss: 1.5666 - accuracy: 0.6013 - val_loss: 0.9039 - val_accuracy: 0.8308
Epoch 2/10
24/24 [==============================] - 69s 3s/step - loss: 0.7478 - accuracy: 0.8320 - val_loss: 0.5985 - val_accuracy: 0.8600
Epoch 3/10
24/24 [==============================] - 69s 3s/step - loss: 0.5355 - accuracy: 0.8747 - val_loss: 0.5027 - val_accuracy: 0.8660
Epoch 4/10
24/24 [==============================] - 69s 3s/step - loss: 0.4395 - accuracy: 0.8960 - val_loss: 0.4568 - val_accuracy: 0.8772
Epoch 5/10
24/24 [==============================] - 69s 3s/step - loss: 0.3754 - accuracy: 0.9080 - val_loss: 0.4289 - val_accuracy: 0.8780
Epoch 6/10
24/24 [==============================] - 69s 3s/step - loss: 0.3278 - accuracy: 0.9293 - val_loss: 0.4094 - val_accuracy: 0.8824
Epoch 7/10
24/24 [==============================] - 69s 3s/step - loss: 0.28

In [ ]:
model_r=model_creator(resnet, 10)
model_r.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
            optimizer=tf.keras.optimizers.Adam(),
            metrics=["accuracy"],
            )
history_r=model_r. fit(train_data,
            epochs=10,
            batch_size=32,
            validation_data=(test_data),
            callbacks=[tensorboard_callback("tensorboard", "resnet_v2")]
            )

saving TensorBoard files to tensorboard/resnet_v2/2022-12-07 08:25:59.588055
Epoch 1/10
24/24 [==============================] - 27s 1s/step - loss: 1.9815 - accuracy: 0.3240 - val_loss: 1.3481 - val_accuracy: 0.5988
Epoch 2/10
24/24 [==============================] - 20s 882ms/step - loss: 1.0527 - accuracy: 0.7240 - val_loss: 0.9188 - val_accuracy: 0.7436
Epoch 3/10
24/24 [==============================] - 20s 870ms/step - loss: 0.7240 - accuracy: 0.8200 - val_loss: 0.7441 - val_accuracy: 0.7972
Epoch 4/10
24/24 [==============================] - 21s 881ms/step - loss: 0.5626 - accuracy: 0.8760 - val_loss: 0.6790 - val_accuracy: 0.8016
Epoch 5/10
24/24 [==============================] - 21s 883ms/step - loss: 0.4612 - accuracy: 0.8920 - val_loss: 0.6255 - val_accuracy: 0.8132
Epoch 6/10
24/24 [==============================] - 25s 1s/step - loss: 0.3954 - accuracy: 0.9293 - val_loss: 0.5861 - val_accuracy: 0.8196
Epoch 7/10
24/24 [==============================] - 20s 865ms/step - lo

In [ ]:
!tensorboard dev upload --logdir ./tensorboard/\
--name "EfficientNetB0, B7 vs ResNetV2"\
--one_shot


New experiment created. View your TensorBoard at: https://tensorboard.dev/experiment/APa7Pph5SMi9k2I0EVyw2Q/

[2022-12-07T09:05:32] Started scanning logdir.
W1207 09:05:35.347084 139761123952512 uploader.py:1092] Blob too large; skipping.  Size 13761548 exceeds limit of 10485760 bytes.
[2022-12-07T09:05:37] Total uploaded: 180 scalars, 0 tensors, 2 binary objects (5.5 MB)
Total skipped: 1 binary objects (13.1 MB)
[2022-12-07T09:05:37] Done scanning logdir.


Done. View your TensorBoard at https://tensorboard.dev/experiment/APa7Pph5SMi9k2I0EVyw2Q/


The plots of comparison can be veiwed [here](https://tensorboard.dev/experiment/APa7Pph5SMi9k2I0EVyw2Q/#scalars)

**Fine tuning** EfficientNet B7

In [1]:
import zipfile

In [2]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_all_data.zip
zip_ref=zipfile.ZipFile("10_food_classes_all_data.zip","r")
zip_ref.extractall()
zip_ref.close()

--2022-12-07 15:26:38--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_all_data.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.12.128, 74.125.68.128, 142.250.4.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.12.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 519183241 (495M) [application/zip]
Saving to: ‘10_food_classes_all_data.zip’

10_food_classes_all 100%[===================>] 495.13M  19.8MB/s    in 26s     

2022-12-07 15:27:04 (19.3 MB/s) - ‘10_food_classes_all_data.zip’ saved [519183241/519183241]



In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir="10_food_classes_all_data/train"
test_dir="10_food_classes_all_data/test"
train_datagen= ImageDataGenerator(rescale=1/255)
test_datagen= ImageDataGenerator(rescale= 1/255)
train_data= train_datagen.flow_from_directory(directory=train_dir,
                                              #class_mode="sparse",
                                              target_size=(300,300),
                                              )
test_data=test_datagen.flow_from_directory(directory=test_dir,
                                           #class_mode="sparse",
                                           target_size=(300,300),
                                           )

Found 7500 images belonging to 10 classes.
Found 2500 images belonging to 10 classes.


In [13]:
import tensorflow as tf
import tensorflow_hub as hub

def model_creator(model_url, num_class=10):
  feature_extractor=hub.KerasLayer(model_url,
                                    #trainable= True,
                                    )
  model=tf.keras.Sequential([ feature_extractor,
                              tf.keras.layers.Dense(num_class, activation="softmax"),
                            ])
  return model

In [ ]:
model_eb7=model_creator("https://tfhub.dev/tensorflow/efficientnet/b7/feature-vector/1", 10)
model_eb7.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
            optimizer=tf.keras.optimizers.Adam(),
            metrics=["accuracy"],
            )
history_eb7=model_eb7.fit(train_data,
            epochs=10,
            batch_size=32,
            validation_data=(test_data),
            #callbacks=[tensorboard_callback("tensorboard", "efficientnet_b7")]
            )

Epoch 1/10
235/235 [==============================] - 252s 960ms/step - loss: 0.6422 - accuracy: 0.8240 - val_loss: 0.3252 - val_accuracy: 0.9088
Epoch 2/10
235/235 [==============================] - 222s 947ms/step - loss: 0.3709 - accuracy: 0.8880 - val_loss: 0.2742 - val_accuracy: 0.9204
Epoch 3/10
235/235 [==============================] - 223s 950ms/step - loss: 0.3127 - accuracy: 0.9032 - val_loss: 0.2592 - val_accuracy: 0.9184
Epoch 4/10
235/235 [==============================] - 222s 947ms/step - loss: 0.2783 - accuracy: 0.9181 - val_loss: 0.2442 - val_accuracy: 0.9236
Epoch 5/10
235/235 [==============================] - 222s 946ms/step - loss: 0.2496 - accuracy: 0.9247 - val_loss: 0.2471 - val_accuracy: 0.9224
Epoch 6/10
235/235 [==============================] - 223s 948ms/step - loss: 0.2292 - accuracy: 0.9316 - val_loss: 0.2355 - val_accuracy: 0.9260
Epoch 7/10
235/235 [==============================] - 223s 948ms/step - loss: 0.2091 - accuracy: 0.9433 - val_loss: 0.2367 -